# Library

In [7]:
import pandas as pd
import numpy as np

## [로드] 컬럼명 데이터

In [8]:
col_df = pd.read_table('data/컬럼명.txt', sep=',')
col= col_df.columns

In [9]:
col

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명', '차종_제원정보',
       '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력',
       '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류'],
      dtype='object')

## [로드] 파일2

In [10]:
# 53.9s
df = pd.read_table('data/220819_22.06월기준_4등급.txt', encoding='cp949', header=40, names=col)
df.info()

C:\Users\kbj\AppData\Local\Temp\ipykernel_6288\497985372.py:2: DtypeWarning: Columns (14,17,29,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_table('data/220819_22.06월기준_4등급.txt', encoding='cp949', header=40, names=col)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528586 entries, 0 to 1528585
Data columns (total 40 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   차량관리번호       1528586 non-null  object 
 1   차량등록번호       1528586 non-null  object 
 2   본거지법정동코드     1528586 non-null  int64  
 3   소유자구분        1528586 non-null  object 
 4   차종_등록정보      1528584 non-null  object 
 5   용도           1528586 non-null  object 
 6   차대번호         1528586 non-null  object 
 7   최초등록일자       1528586 non-null  int64  
 8   차량연식         1528586 non-null  int64  
 9   차량제작일자       1358925 non-null  float64
 10  검사유효일        1528574 non-null  float64
 11  제원관리번호       1528586 non-null  object 
 12  배출가스인증번호     1405585 non-null  object 
 13  배출가스등급       1528586 non-null  int64  
 14  저공해조치종류      195 non-null      object 
 15  저공해조치구조변경YN  1528586 non-null  object 
 16  차량말소YN       1528586 non-null  object 
 17  취득일자         1435514 non-null  object 
 18  취득

In [11]:
f = df.groupby('연료', as_index=False)['차대번호'].count().sort_values('차대번호', ascending=False)
f

,연료,차대번호
2,경유,1167610
5,휘발유,334517
1,LPG(액화석유가스),26419
3,기타연료,38
0,CNG(압축천연가스),1
4,알코올,1


In [12]:
f.columns = ['연료', '4등급']
f

,연료,4등급
2,경유,1167610
5,휘발유,334517
1,LPG(액화석유가스),26419
3,기타연료,38
0,CNG(압축천연가스),1
4,알코올,1


In [14]:
df[df['연료'] == '기타연료']

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,배기량,승차인원,차량길이,차량너비,차량높이,원산지명,삭제YN,제원승인일자,구동형식,변속기종류
38295,KNAGB2212KS212905-01,경북1느7916,4729011700,주민,승용,자가용,KNAGB2212KS212905,19891006,1989,NaN,...,1498.0,5,4430.0,1705.0,1405.0,대한민국,N,19880922,NaN,수동
73224,KLATF1961KB208976-01,대구1거4502,2714010900,주민,승용,자가용,KLATF1961KB208976,19890710,1989,NaN,...,1598.0,5,4394.0,1663.0,1362.0,대한민국,N,19870902.0,NaN,수동
103131,12851300018EORN-01,인천30고8941,4711310700,주민,승용,자가용,12851300018EORN,19870730,2001,NaN,...,2759.0,0,4690.0,1690.0,1440.0,미국,N,19961004,NaN,수동
105842,KNAGB2212KS205388-01,대구1머3595,2711010100,주민,승용,자가용,KNAGB2212KS205388,19890720,1989,NaN,...,1498.0,5,4430.0,1705.0,1405.0,대한민국,N,19880922,NaN,수동
113699,KLASR19F1KB302533-01,대구1머1600,2723010800,주민,승용,자가용,KLASR19F1KB302533,19890621,1989,NaN,...,1979.0,5,4934.0,1720.0,1420.0,대한민국,N,19881122,NaN,수동
121116,KNAGB2212KS212056-01,경기3커6299,4161025329,주민,승용,자가용,KNAGB2212KS212056,19890919,1989,NaN,...,1498.0,5,4430.0,1705.0,1405.0,대한민국,N,19880922.0,NaN,수동
150211,KMHSF21APJU300507-01,대구1러1984,2726011100,주민,승용,자가용,KMHSF21APJU300507,19881219,1988,NaN,...,1499.0,5,4556.0,1716.0,1381.0,대한민국,N,NaN,NaN,수동
162707,IMECM53UOLA620230-01,경기3모4667,4157010700,주민,승용,자가용,IMECM53UOLA620230,19900512,1990,NaN,...,2986.0,5,488.0,180.0,138.0,미국,N,19891223,NaN,수동
373733,WDB12322310164608-01,서울1트5997,4580039022,주민,승용,자가용,WDB12322310164608,19881109,2001,NaN,...,2299.0,5,4720.0,1780.0,1430.0,독일,N,19920220,NaN,수동
431838,KNAGB2212KS201608-01,대구3수6467,2771026200,주민,승용,자가용,KNAGB2212KS201608,19890420,1989,NaN,...,1498.0,5,4430.0,1705.0,1405.0,대한민국,N,19880922,NaN,수동


## [excel] 연료별 통계

In [13]:
f.to_excel('analysis/[G4]연료별_차대번호_통계.xlsx', index=False, encoding='utf-8')

C:\Users\kbj\AppData\Roaming\Python\Python39\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


# 지역별 수량 통계

## [로드] 법정동코드

In [18]:
code = pd.read_excel('data/지역구분코드(수정).xlsx')
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229 entries, 0 to 228
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ad1     229 non-null    object
 1   code1   229 non-null    int64 
 2   ad2     229 non-null    object
 3   code2   229 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 7.3+ KB


In [19]:
code = code[['ad1', 'code1']].drop_duplicates(['ad1'], keep='first').reset_index(drop=True)
code

,ad1,code1
0,서울특별시,11
1,부산광역시,26
2,대구광역시,27
3,인천광역시,28
4,광주광역시,29
5,대전광역시,30
6,울산광역시,31
7,세종특별자치시,36
8,경기도,41
9,강원도,42


## 법정동 코드와 매칭

In [20]:
df['본거지법정동코드'].head()

0    4157010100
1    1165010700
2    2638010200
3    2614010100
4    1126010500
Name: 본거지법정동코드, dtype: int64

In [21]:
df['본거지법정동코드'].isnull().sum()

0

In [22]:
df['지역코드'] = df['본거지법정동코드'].astype('str').str[:2]
df['지역코드'] = df['지역코드'].astype('int')

In [23]:
df['지역코드'].head()

0    41
1    11
2    26
3    26
4    11
Name: 지역코드, dtype: int32

In [24]:
code['code1'].head()

0    11
1    26
2    27
3    28
4    29
Name: code1, dtype: int64

In [25]:
df = df.merge(code, left_on='지역코드', right_on='code1', how='left')
df[['지역코드', 'ad1']].head()

,지역코드,ad1
0,41,경기도
1,11,서울특별시
2,26,부산광역시
3,26,부산광역시
4,11,서울특별시


In [26]:
df['ad1'].isnull().sum()

0

In [27]:
l = df.groupby(['ad1'], as_index=False)['차대번호'].count()
l

,ad1,차대번호
0,강원도,349557
1,경기도,2796166
2,경상남도,839431
3,경상북도,620475
4,광주광역시,317327
5,대구광역시,547738
6,대전광역시,321244
7,부산광역시,665142
8,서울특별시,1430746
9,세종특별자치시,91006


In [30]:
l.columns = ['지역', '2등급']
l

,지역,2등급
0,강원도,349557
1,경기도,2796166
2,경상남도,839431
3,경상북도,620475
4,광주광역시,317327
5,대구광역시,547738
6,대전광역시,321244
7,부산광역시,665142
8,서울특별시,1430746
9,세종특별자치시,91006


### [excel] 지역별 수량 통계

In [31]:
l.to_excel('analysis/[G2]지역별_차대번호_통계.xlsx', index=False, encoding='utf-8')

C:\Users\kbj\AppData\Roaming\Python\Python39\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


# 코드 마지막